In [ ]:
from pynq import Overlay, allocate
from time import perf_counter_ns
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [ ]:
overlay = Overlay('lcs.bit')
lcs_ip = overlay.lcs

In [ ]:
CONTROL = 0x00
IMAGE_OUT = 0x10
IMAGE_IN = 0x18
IMAGE_LENGTH = 0x20
LOW_THRESHOLD = 0x28
HIGH_THRESHOLD = 0x30
LOW_NEW_THRESHOLD = 0x38
HIGH_NEW_THRESHOLD = 0x40
MAX_VALUE = 0x48

In [ ]:
image_path = 'lena.tif'
image = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)

plt.axis('off')
plt.imshow(image)

In [ ]:
grey_image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)

plt.axis('off')
plt.imshow(grey_image, cmap='gray')

In [ ]:
flat_image = grey_image.flatten().astype(np.uint32)

In [ ]:
image_in = allocate(shape=flat_image.shape, dtype=np.uint32, cacheable=0)
image_out = allocate(shape=flat_image.shape, dtype=np.uint32, cacheable=0)

In [ ]:
image_in[:] = flat_image

lcs_ip.write(IMAGE_IN, image_in.physical_address)
lcs_ip.write(IMAGE_OUT, image_out.physical_address)

lcs_ip.write(IMAGE_LENGTH, flat_image.size)
lcs_ip.write(LOW_THRESHOLD, 140)
lcs_ip.write(HIGH_THRESHOLD, 200)
lcs_ip.write(LOW_NEW_THRESHOLD, 70)
lcs_ip.write(HIGH_NEW_THRESHOLD, 230)
lcs_ip.write(MAX_VALUE, 255)

In [ ]:
start = perf_counter_ns()

lcs_ip.write(CONTROL, 1 << 0)
while lcs_ip.read(CONTROL) != (1 << 2):
    pass

stop = perf_counter_ns()

print(F'Elapsed time: {(stop - start) / 1e6} ms')

In [ ]:
processed_flat_image = image_out

processed_image = processed_flat_image.reshape(grey_image.shape).astype(np.uint8)

plt.axis('off')
plt.imshow(processed_image, cmap='gray')